<a href="https://colab.research.google.com/github/matsu641/DL-practice/blob/main/lecture10_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第10回講義 宿題



## 課題
自己教師あり学習を用いて事前学習を行い，得られた表現をLinear probingで評価してみましょう．  
ネットワークの形などに制限はとくになく，今回のLessonで扱った内容以外の工夫も組み込んでもらって構いません．   



### 目標値
なし
- 自己教師あり学習の手法によっては計算リソースによって性能が大きく変わるため，目標精度は設定していません．
- ただし以下の工夫を行うことで計算リソースが少なくとも，長い学習を分割して行うことができます．  
    - model，optimizer, schedulerを一定エポックで保存して，読み込むことで学習を再開することができます．
    - 演習のようにschedulerを実装した場合は保存は必要なく，同じ引数でインスタンスを作成して`__call__`の際に与えるepochを学習の続きから与えれば動作します．  
    - 参考: https://pytorch.org/tutorials/beginner/saving_loading_models.html



### ルール
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- 自己教師あり学習では以下のセルで指定されている`x_train`以外の学習データは用いないでください．
- Linear probingの際には`x_train`, `t_train`以外の学習データは用いないでください．


### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第10回 表現学習と自己教師あり学習」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第10回 表現学習と自己教師あり学習 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．


### 評価方法
- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 作業ディレクトリを指定
work_dir = 'drive/MyDrive/Colab Notebooks/DLBasics2025_colab'

### データの読み込み（このセルは修正しないでください）

In [ ]:
import random

import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

#学習データ
x_train = np.load(work_dir + '/Lecture10/data/x_train.npy')
t_train = np.load(work_dir + '/Lecture10/data/t_train.npy')

#テストデータ
x_test = np.load(work_dir + '/Lecture10/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        data = x_train.astype('float32')
        self.x_train = []
        for i in range(data.shape[0]):
            self.x_train.append(Image.fromarray(np.uint8(data[i])))
        self.t_train = t_train
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        return self.transform(self.x_train[idx]), torch.tensor(t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        data = x_test.astype('float32')
        self.x_test = []
        for i in range(data.shape[0]):
            self.x_test.append(Image.fromarray(np.uint8(data[i])))
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_test)

    def __getitem__(self, idx):
        return self.transform(self.x_test[idx])

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### データローダの準備  

In [ ]:
val_size = 3000
train_data, valid_data = torch.utils.data.random_split(trainval_data, [len(trainval_data) - val_size, val_size])

train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)]
)
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)]
)

trainval_data.transform = train_transform
test_data.transform = test_transform

batch_size = 128

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    valid_data,
    batch_size=batch_size,
    shuffle=False
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

### 自己教師あり学習の実装
MAEを利用することを想定していますが，他の自己教師あり学習を利用していただいて構いません．   

In [ ]:
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


fix_seed(seed=42)


def random_indexes(size):
    """
    パッチをランダムに並べ替えるためのindexを生成する関数．

    Argument
    --------
    size : int
        入力されるパッチの数（系列長Nと同じ値）．
    """
    forward_indexes = np.arange(size)  # 0からsizeまでを並べた配列を作成
    np.random.shuffle(forward_indexes)  # 生成した配列をシャッフルすることで，パッチの順番をランダムに決定
    backward_indexes = np.argsort(forward_indexes)  # 並べ替えたパッチをもとの順番に戻すためのidx

    return forward_indexes, backward_indexes


def take_indexes(sequences, indexes):
    """
    パッチを並べ替えるための関数．

    Argument
    --------
    sequences : torch.Tensor
        入力画像をパッチ分割したデータ．(B, N, dim)の形状をしている．
    indexes : np.ndarray
        並べ替えるために利用するindex．
        random_indexesで生成したforward_indexesかbackward_indexesが入ることが想定されている．
    """
    return torch.gather(sequences, dim=1, index=indexes.unsqueeze(2).repeat(1, 1, sequences.shape[-1]))


class Attention(nn.Module):
    # WRITE ME


class FFN(nn.Module):
    # WRITE ME


class Block(nn.Module):
    # WRITE ME


class PatchEmbedding(nn.Module):
    # WRITE ME


class PatchShuffle(nn.Module):
    # WRITE ME


class MAE_Encoder(nn.Module):
    # WRITE ME


class MAE_Decoder(nn.Module):
    # WRITE ME


class MAE_ViT(nn.Module):
    # WRITE ME


device = "cuda" if torch.cuda.is_available() else "cpu"
model = # WRITE ME
epochs = # WRITE ME
lr = # WRITE ME
batch_size = # WRITE ME
optimizer = # WRITE ME

### 事前学習（自己教師あり学習）

In [ ]:
for epoch in range(epochs):
    # WRITE ME

### Linear probing

In [ ]:
val_size = 3000
train_data, valid_data = torch.utils.data.random_split(trainval_data, [len(trainval_data) - val_size, val_size])

train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)]
)
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)]
)

trainval_data.transform = train_transform
test_data.transform = test_transform

batch_size = 128

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
class Classifier(nn.Module):
    # WRITE ME


device = "cuda" if torch.cuda.is_available() else "cpu"

encoder = # WRITE ME
model = # WRITE ME
epochs = # WRITE ME
lr = # WRITE ME
batch_size = # WRITE ME
optimizer = # WRITE ME

In [ ]:
for epoch in range(epochs):
    # WRITE ME

In [ ]:
model.eval()

t_pred = []
for x in dataloader_test:
    x = x.to(device)
    y = model(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()
    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv(work_dir + '/Lecture10/submission_pred.csv', header=True, index_label='id')